In [86]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [87]:
# Data preparation
documents = [
    "I love reading books. Books are a great source of knowledge.",
    "The internet is full of information. We can find answers to our questions online.",
    "Newspapers provide news from around the world. They keep us updated.",
    "Studying hard is important for academic success. It helps in achieving good grades."
]
summaries = [
    "Books are a great source of knowledge.",
    "The internet provides answers to our questions.",
    "Newspapers keep us updated with news.",
    "Studying hard leads to academic success."
]

In [88]:
# Tokenize the documents and summaries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer_doc = Tokenizer(num_words=1000)
tokenizer_doc.fit_on_texts(documents)
sequences_doc = tokenizer_doc.texts_to_sequences(documents)

tokenizer_summ = Tokenizer(num_words=1000)
tokenizer_summ.fit_on_texts(summaries)
sequences_summ = tokenizer_summ.texts_to_sequences(summaries)

In [89]:
# Pad sequences to the same length
max_document_length = max(len(seq) for seq in sequences_doc)
max_summary_length = max(len(seq) for seq in sequences_summ)

padded_sequences_doc = pad_sequences(sequences_doc, maxlen=max_document_length)
padded_sequences_summ = pad_sequences(sequences_summ, maxlen=max_summary_length)

In [90]:
# Define the encoder-decoder model
encoder_input = Input(shape=(max_document_length,1))
encoder = LSTM(128)(encoder_input)

decoder_input = Input(shape=(max_summary_length,1))
decoder = LSTM(128, return_sequences=True)(decoder_input)
decoder_output = Dense(tokenizer_summ.num_words, activation='softmax')(decoder)

model = Model([encoder_input, decoder_input], decoder_output)


In [91]:
# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit([padded_sequences_doc, padded_sequences_summ], np.expand_dims(padded_sequences_summ, -1), batch_size=25, epochs=70)

Epoch 1/70
1/1 [==============================] - 3s 3s/step - loss: 6.8882
Epoch 2/70
1/1 [==============================] - 0s 18ms/step - loss: 6.8180
Epoch 3/70
1/1 [==============================] - 0s 20ms/step - loss: 6.7468
Epoch 4/70
1/1 [==============================] - 0s 17ms/step - loss: 6.6739
Epoch 5/70
1/1 [==============================] - 0s 18ms/step - loss: 6.5983
Epoch 6/70
1/1 [==============================] - 0s 19ms/step - loss: 6.5190
Epoch 7/70
1/1 [==============================] - 0s 25ms/step - loss: 6.4350
Epoch 8/70
1/1 [==============================] - 0s 22ms/step - loss: 6.3453
Epoch 9/70
1/1 [==============================] - 0s 19ms/step - loss: 6.2491
Epoch 10/70
1/1 [==============================] - 0s 19ms/step - loss: 6.1455
Epoch 11/70
1/1 [==============================] - 0s 17ms/step - loss: 6.0340
Epoch 12/70
1/1 [==============================] - 0s 21ms/step - loss: 5.9141
Epoch 13/70
1/1 [==============================] - 0s 18ms/step

In [92]:
# Generate summaries for new documents
new_documents = [
    "Reading is my favorite hobby. I enjoy exploring different genres of books.",
    "The internet has revolutionized the way we access information.",
    "Newspapers play a crucial role in journalism and democracy.",
    "Studying smart is more important than studying hard for academic success."
]
new_sequences_doc = tokenizer_doc.texts_to_sequences(new_documents)
padded_new_sequences_doc = pad_sequences(new_sequences_doc, maxlen=max_document_length)

encoder_input_data = padded_new_sequences_doc
decoder_input_data = np.zeros((len(new_documents), max_summary_length))

In [93]:
# Generate summaries using the trained model
decoded_sequences = model.predict([encoder_input_data, decoder_input_data])

1/1 [==============================] - 0s 474ms/step


In [94]:
# Convert predicted sequences to text summaries
predicted_summaries = []
for decoded_sequence in decoded_sequences:
    summary = []
    for token_id in decoded_sequence.argmax(axis=1):
        word = tokenizer_summ.index_word[token_id]
        if word is None or word == '<pad>':
            break
        summary.append(word)
    predicted_summaries.append(' '.join(summary))

In [95]:
# Print the predicted summaries
for document, summary in zip(new_documents, predicted_summaries):
    print('Document:', document)
    print('Predicted Summary:', summary)
    print()

Document: Reading is my favorite hobby. I enjoy exploring different genres of books.
Predicted Summary: are a to to to to to

Document: The internet has revolutionized the way we access information.
Predicted Summary: are a to to to to to

Document: Newspapers play a crucial role in journalism and democracy.
Predicted Summary: are a to to to to to

Document: Studying smart is more important than studying hard for academic success.
Predicted Summary: are a to to to to to

